In [55]:
faqs = """Deep learning is a powerful subset of machine learning inspired by the structure and function of the human brain. It uses artificial neural networks with multiple layers — hence “deep” — to automatically learn hierarchical representations of data. Unlike traditional machine learning, which often requires manual feature engineering, deep learning models extract features directly from raw input (images, text, audio, etc.), making them highly scalable and effective for complex tasks.

At its core, deep learning relies on architectures like Convolutional Neural Networks (CNNs) for image recognition, Recurrent Neural Networks (RNNs) and Transformers for sequential data like text or speech, and Generative Adversarial Networks (GANs) for creating synthetic data. These models are trained using large datasets and optimized via backpropagation and gradient descent to minimize prediction errors.

Deep learning powers today’s most advanced AI applications: facial recognition, autonomous vehicles, real-time language translation, medical diagnosis, recommendation systems, and even creative tools like AI-generated art and music. Its success is fueled by three key factors: massive labeled datasets, powerful hardware (GPUs/TPUs), and algorithmic innovations (e.g., attention mechanisms, batch normalization, dropout).

Despite its strengths, deep learning has challenges. It demands significant computational resources and data, lacks interpretability (“black box” problem), and can inherit biases from training data. Models may also overfit or fail catastrophically on edge cases.

Still, deep learning continues to evolve rapidly, with research pushing toward more efficient, robust, and generalizable models — including self-supervised learning, few-shot learning, and neural-symbolic integration. Frameworks like TensorFlow, PyTorch, and JAX have democratized access, enabling researchers and developers worldwide to build and deploy state-of-the-art models.

In essence, deep learning has revolutionized AI by enabling machines to perceive, reason, and generate with human-like capability — transforming industries and redefining what’s possible in technology. Its journey is far from over, with future breakthroughs poised to unlock even greater intelligence and autonomy in machines.
"""

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [57]:
tokenizer = Tokenizer()

In [58]:
tokenizer.fit_on_texts([faqs])
# we can send multiple texts so input is in list

In [59]:
tokenizer.word_index

{'and': 1,
 'learning': 2,
 'deep': 3,
 'to': 4,
 'data': 5,
 'models': 6,
 'like': 7,
 'of': 8,
 'neural': 9,
 'networks': 10,
 'with': 11,
 '—': 12,
 'for': 13,
 'its': 14,
 'is': 15,
 'by': 16,
 'the': 17,
 'from': 18,
 'ai': 19,
 'in': 20,
 'powerful': 21,
 'machine': 22,
 'human': 23,
 'it': 24,
 'text': 25,
 'on': 26,
 'recognition': 27,
 'or': 28,
 'datasets': 29,
 'even': 30,
 'art': 31,
 'has': 32,
 'enabling': 33,
 'machines': 34,
 'a': 35,
 'subset': 36,
 'inspired': 37,
 'structure': 38,
 'function': 39,
 'brain': 40,
 'uses': 41,
 'artificial': 42,
 'multiple': 43,
 'layers': 44,
 'hence': 45,
 '“deep”': 46,
 'automatically': 47,
 'learn': 48,
 'hierarchical': 49,
 'representations': 50,
 'unlike': 51,
 'traditional': 52,
 'which': 53,
 'often': 54,
 'requires': 55,
 'manual': 56,
 'feature': 57,
 'engineering': 58,
 'extract': 59,
 'features': 60,
 'directly': 61,
 'raw': 62,
 'input': 63,
 'images': 64,
 'audio': 65,
 'etc': 66,
 'making': 67,
 'them': 68,
 'highly': 69,

In [60]:
input_sequences = []
for sentence in faqs.split('\n'): #every line has one sentence so split based on new line
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
  # every sentence will be shown as a number vector in a list
  # we use 0th item of list as it has only one in list so only one vector be shown
  for i in range(1,len(tokenized_sentence)):# range 1 to len of sentence
    input_sequences.append(tokenized_sentence[:i+1])
# as [:i+1] for i=1 will extract from 0 to 2 and so on as atleast 2 words/sent req in input. it will extract start from 2 words per sentence and so on till end of all sentences

In [80]:
len(input_sequences)

310

In [62]:
# zero padding req to equalize all sentences, first max len sentence's words req
max_len = max([len(x) for x in input_sequences])
max_len

70

In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [76]:
# unique words in our text
len(tokenizer.word_index)

218

In [64]:
# store our input and output labells
X = padded_input_sequences[:,:-1] # all rows and cols except last col
y = padded_input_sequences[:,-1]
# all rows and last col, will give only one sclar value from each sentence

In [65]:
print(X.shape)
print(y.shape)

(310, 69)
(310,)


In [66]:
# convert output label to OHE number
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=len(tokenizer.word_index)+1)
# len of word, one added as One hot encoding start from 0 while word start from 1. 218 words in our text

In [67]:
y.shape

(310, 219)

In [68]:
# every output start from 0 as no 0th word and end at 219 word which is at 220th place
y[:5,]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [75]:
model = Sequential()
model.add(Embedding(input_dim=219, output_dim=100))
#last word y so length one less, 219 vocabulary due to addition of 0, otherwise 218 which are unique words in our text
#output_dim is hyperparameter and below LSTM node 150 is too
model.add(LSTM(150))
model.add(Dense(219, activation='softmax'))

#

In [78]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [79]:
model.fit(X,y, epochs=100)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.3263 - loss: 2.5465
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.4060 - loss: 2.3854
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.4375 - loss: 2.2969
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step - accuracy: 0.4877 - loss: 2.2225
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 232ms/step - accuracy: 0.4942 - loss: 2.1453
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - accuracy: 0.5542 - loss: 2.0777
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.5500 - loss: 2.0561
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.6188 - loss: 1.9521
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.6180 - loss: 1.8625
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.6276 - loss: 1.8347
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.6838 - loss: 1.7817
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1

In [81]:
# summary can be seen only after fitting the model
model.summary()
# 219 no input x 100 nodes = 21900 param
#formula for LSTM 4 * ((num_units * num_units) + (num_units * input_dim) + num_units)
# The formula includes multiplying by 4 because an LSTM cell has four gates
# (input, forget, output, and a gate that computes the candidate hidden state),
# and each gate has its own set of weights and biases.
# 4 * ((150 * 150) + (150 * 100) + 150)=150600
# dense layer 150 input x 219 nodes + 283 biases= 33069

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 69, 100)        │        21,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 219)            │        33,069 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 616,709 (2.35 MB)

 Trainable params: 205,569 (803.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 411,140 (1.57 MB)

In [84]:
import numpy as np
# predict on any text present in our training data
text = 'deep learning' # we have to predict next word to text
# tokenize
token_text = tokenizer.texts_to_sequences([text])[0]
print(token_text)
# padding, equalize length to model's req input
padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
print(padded_token_text)
#predict, it is numpy array having prob of each word
model.predict(padded_token_text).shape
# finding highest prob word which is our pred i.e next word to our text
output = np.argmax(model.predict(padded_token_text))
print(output)

[3, 2]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
15


In [85]:
# to find next word in word form
for word, index in tokenizer.word_index.items():
  if index == output:
    print(word)

is


In [91]:
# if want to predict five words i.e next four words also with pred word
text = 'deep learning has revolutionized'

for i in range(10):
  token_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
  output = np.argmax(model.predict(padded_token_text))
  for word, index in tokenizer.word_index.items():
    if index == output:
      text = text + ' ' + word
      print(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
deep learning has revolutionized ai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
deep learning has revolutionized ai by
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
deep learning has revolutionized ai by enabling
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
deep learning has revolutionized ai by enabling machines
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
deep learning has revolutionized ai by enabling machines to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
deep learning has revolutionized ai by enabling machines to perceive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
deep learning has revolutionized ai by enabling machines to perceive reason
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
deep learning has revolutionized ai by enabling machines to perceive reason and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
deep learning has revolutionized ai by enabling machines to perceive reason and generate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
deep learning has revolutionized ai by enabling machines to percei